# LMCX-4

Updating the dataset: A044010200000
* Wavelength shift between combined spectrum and individual segments is not an error.  Within an NVO file, segments are shifted to match the guide channel (either LiF 1 or LiF 2) between 1045 and 1070 A.
* Comparison of LiF 2A and LiF 1B spectra shows no evidence of a worm between 1090 and 1120 A.  There is a strong worm at the long-wavelength end of the LiF 1B spectrum.  Used LiF 2A to rescale LiF 1B between 1180 and 1190 A.

COMMENT File updated 21 Mar 2022 <br>
COMMENT 1080-1190 scaled by 1.278242588043213 to correct for worm

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.io import fits
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import extract_region

# Specify plot paramters.
quantity_support()  # for getting units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Normalize all spectra, so fluxes are of order unity.
norm = 1E13

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Read header keywords.

filename = 'a044010200000all4ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# Guide channel is LiF 1.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Let's compare the two long-wavelength channels:

f, ax = plt.subplots()  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF1B') 
ax.legend() 

# LiF 1B suffers from an obvious worm.  We will rescale it to match LiF 2A.

In [ ]:
# Select a spectral region.

region = SpectralRegion(1179*u.AA, 1181*u.AA)
sub_lif1b = extract_region(lif1b, region)
sub_lif2a = extract_region(lif2a, region)

# Compute ratio of their fluxes.

scale_lif1b = sub_lif2a.mean()/sub_lif1b.mean()
print ('Scale LiF 1b by', scale_lif1b)

In [ ]:
# Scale the smoothed spectra.

lif1b_bsmooth *= scale_lif1b

In [ ]:
# Plot rescaled spectrum.

f, ax = plt.subplots()  
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF1B') 
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF2A') 
ax.legend()

# LiF 1B is now OK at the longest wavelengths.

In [ ]:
# Compare LiF 2A with a STIS spectrum of the same star.

filename = 'o49wa5010_x1d.fits'
print ('Target:  ', fits.getval(filename, 'TARGNAME'))
print ('Aperture:', fits.getval(filename, 'APERTURE'))
stis = Spectrum1D.read(filename, format="HST/STIS")
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
#ax.plot(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.set_xlim([1120,1200])
ax.set_ylim([0,10])
ax.legend()

# The fluxes match, but there is an interesting difference in the shape of the 1175 A feature.

In [ ]:
# Apply LiF 1B correction to the NVO file.

filename = 'a044010200000nvo4ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Scale LiF 1B.
g = np.where(data['wave'] > 1180)
data['flux'][g] *= float(scale_lif1b)
data['error'][g] *= float(scale_lif1b)

hdr['comment'] = ''
hdr['comment'] = 'File updated 21 Mar 2022'
hdr['comment'] = '1080-1190 scaled by ' + str(scale_lif1b) + ' to correct for worm'

f.writeto('level0_a044010200000nvo4ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'a044010200000nvo4ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 

filename = 'level0_a044010200000nvo4ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.legend()
ax2.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,1E-12])